In [1]:
# designed to be run after 03-clinical_variables_final. this notebook does some data cleaning/processing. run before -___ notebook.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np
import glob


from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime
%reload_ext autotime

In [3]:
#cohort import
from pathlib import Path
os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'

date='25012019'
final_pt_df2 = pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/25012019_final_pt_df2_v.csv', index_col=0) #final cohort for PA 

patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 65.6 ms


In [4]:
#adding this in fow now, will eventually want to switch this to the 03-clinical_variables_final
import psycopg2
import getpass
conn = psycopg2.connect(dbname="mimic", user="student", host="luolabwkstn01.fsm.northwestern.edu", port="5432",
                        password=getpass.getpass(prompt='Password:'.format()))
cur=conn.cursor()

query_schema = 'SET search_path to ' + "mimiciii" + ';'

Password: ·····


time: 3.68 s


In [5]:
#folder to save files to:
save_path= wd+'/data/processed/PA_project'

def save_df(df, df_name='default'):
    if df_name == 'default':
        df_name= "%s"%(df)
    pd.DataFrame(df).to_csv(Path(save_path+'/%s_%s_prepped.csv' %(date, df_name)))
    del(df)

time: 2.96 ms


In [6]:
#time_variable:

time_var='first_pos_else_neg_ssc' #for origional project time_var = 't_0', pa_project: 'first_pos_else_neg_ssc'

time: 764 µs


In [7]:
list(final_pt_df2)

['subject_id',
 'hadm_id',
 't_0_sc',
 'icustay_id',
 'ab_course',
 't_0',
 't_end_consec',
 'ICU_admit',
 'dod_hosp',
 'dod_ssn',
 'dod_delta',
 'org_list',
 'spec_type_list',
 'Gram_pos',
 'first_pos_else_neg_ssc',
 'sc_result',
 'final_bin']

time: 2.73 ms


# changing my code structure to be a dictionary of dataframes

In [8]:
#importing in all clinical_variable files
date= '25012019'
os.chdir(r'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/PA_project/')
allFiles = glob.glob(os.getcwd() + "/{}_*.csv".format(date))

time: 2.01 ms


In [9]:
allFiles

['/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/PA_project/25012019_ventcategory_df.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/PA_project/25012019_pt_info_df.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/PA_project/25012019_heightfirstday_df.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/PA_project/25012019_sofa_df_72.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/PA_project/25012019_uti_all_72.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/PA_project/25012019_rrt_merged_allpt_df.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/PA_project/25012019_weightfirstday_df.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/PA_project/25012019_vaso_dose_72.csv',
 '/Users/geickelb1/Documents/GitHub/mimici

time: 1.86 ms


In [10]:
#making a dictionary of all my dataframes for easier cycling through

df_list=[]
for element in allFiles:
    df_list.append(element.split('{}_'.format(date))[1].split('.csv')[0]) #making an list of all my dataframes in order they appear in file

dfs = {}
i=0
for name in df_list:
    dfs[name] = pd.read_csv(allFiles[i],  index_col=0)
    i+=1

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 6.55 s


In [11]:
# ##dropping hadm_id from all:
# list1=[]
# for element in df_list:
#     if 'hadm_id' in (list(dfs[element])):
#         list1.append(element)

# for element in list1:
#     dfs[element]= dfs[element].drop('hadm_id', axis=1)

time: 1.11 ms


In [12]:
#all of the column names 
for element in df_list:
    print(element,':',list(dfs[element]))

ventcategory_df : ['icustay_id', 'day', 'value', 'uom', 'first_pos_else_neg_ssc']
pt_info_df : ['subject_id', 'gender']
heightfirstday_df : ['icustay_id', 'height', 'height_chart', 'height_echo', 'uom']
sofa_df_72 : ['subject_id', 'hadm_id', 'icustay_id', 'day', 'sofa', 'respiration', 'pao2fio2_vent_min', 'pao2fio2_novent_min', 'coagulation', 'platelet_min', 'liver', 'bilirubin_max', 'cardiovascular', 'rate_dopamine', 'rate_epinephrine', 'rate_norepinephrine', 'rate_dobutamine', 'meanbp_min', 'cns', 'mingcs', 'renal', 'creatinine_max', 'urineoutput', 'first_pos_else_neg_ssc', 'ICU_admit', 'approx_charttime']
uti_all_72 : ['subject_id', 'hadm_id', 'itemid', 'charttime', 'value', 'valuenum', 'valueuom', 'label', 'fluid', 'category', 'loinc_code', 'icd9_code', 'first_pos_else_neg_ssc', 'delta']
rrt_merged_allpt_df : ['icustay_id', 'first_pos_else_neg_ssc', 'first_charttime', 'rrt', 'uom']
weightfirstday_df : ['icustay_id', 'weight', 'weight_admit', 'weight_daily', 'weight_echoinhosp', 'we

#### Note: Nelson has said we do not need to incorporate Echo data, so this has been removed 10/22/18

In [13]:
# #reading in all of my data that is not limited to 72 hour time window between t_0 and t+72
# #date= '27082018'  #date of the files i will be importing
# date= '25012019'
# ventcategory_df=pd.read_csv(Path(
#     wd+'/data/raw/csv/%s_ventcategory_df.csv' %(date)), index_col=0)

# vaso_dose_72=pd.read_csv(Path(
#     wd+'/data/raw/csv/72_hr_window/%s_vaso_dose_72.csv' %(date)), index_col=0)
# labs_all_nosummary_72=pd.read_csv(Path(
#     wd+'/data/raw/csv/72_hr_window/%s_labs_all_nosummary_72.csv' %(date)), index_col=0)

# weightfirstday_df=pd.read_csv(Path(
#     wd+'/data/raw/csv/72_hr_window/%s_weightfirstday_df.csv' %(date)), index_col=0)
# heightfirstday_df=pd.read_csv(Path(
#     wd+'/data/raw/csv/72_hr_window/%s_heightfirstday_df.csv' %(date)), index_col=0)

# vitals_all_nosummary_72=pd.read_csv(Path(
#     wd+'/data/raw/csv/72_hr_window/%s_vitals_all_nosummary_72.csv' %(date)), index_col=0)

# bg_all_nosummary_72=pd.read_csv(Path(
#     wd+'/data/raw/csv/72_hr_window/%s_bg_all_nosummary_72.csv' %(date)), index_col=0)


# rrt_merged_allpt_df=pd.read_csv(Path(
#     wd+'/data/raw/csv/72_hr_window/%s_rrt_merged_allpt_df.csv' %(date)), index_col=0)
# gcs72_df=pd.read_csv(Path(
#     wd+'/data/raw/csv/72_hr_window/%s_gcs72_df.csv' %(date)), index_col=0)

# sofa_df_72=pd.read_csv(Path(
#     wd+'/data/raw/csv/%s_sofa_df_72.csv' %(date)), index_col=0)

# # echodata_72=pd.read_csv(Path(
# #     wd+'/data/raw/csv/72_hr_window/%s_echodata_72.csv' %(date)), index_col=0)
# # uti_all_72=pd.read_csv(Path(
# #     wd+'/data/raw/csv/72_hr_window/%s_uti_all_72.csv' %(date)), index_col=0)


time: 7.36 ms


In [14]:
##1-29-19 subseting and time windowed moved to 03.01
# def df_subset(df):
#     try:
#         df = df.loc[df.loc[:,'icustay_id'].isin(icustay_id),:]
#     except KeyError or NameError:
#         try:
#             df = df.loc[df.loc[:,'hadm_id'].isin(hadm_id),:]
#         except KeyError or NameError:
#             df = df.loc[df.loc[:,'subject_id'].isin(patients),:]
#     return(df)

# # subsetting each dataframe to only patients in final_patients_df2_v:
# dataframe_list= [
#     ventcategory_df, vaso_dose_72, #echodata_72,
#     labs_all_nosummary_72, weightfirstday_df,
#     heightfirstday_df, vitals_all_nosummary_72,
#     uti_all_72, bg_all_nosummary_72,
#     rrt_merged_allpt_df, gcs72_df, sofa_df_72
# ]

# (
# ventcategory_df, vaso_dose_72, #echodata_72,
# labs_all_nosummary_72, weightfirstday_df,
# heightfirstday_df, vitals_all_nosummary_72,
# uti_all_72, bg_all_nosummary_72,
# rrt_merged_allpt_df, gcs72_df, sofa_df_72
# ) = (df_subset(df) for df in dataframe_list)

time: 6.43 ms


In [15]:
def yn_convert(df, #df in format where each row corresponds to a test, and a patient can have many rows
               label_fill, # value that will be filled to na's
               pt= final_pt_df2,
               time_var='t_0', #
               value_fill=0,
               delta_fill=pd.to_timedelta('0 days'),
               uom_fill='y/n'):

    """
    description: collapses a dataframe where each row corresponds to a test, and a patient can have many rows -> 
    1 row per patient where value is binary variable yes or no a patient has any value within the timewindow (specified in data collection).
    said a different way, for patient this fxn collapses values down to does pt have in time window y/n? 
    """
    
    yn_df = pd.merge(pt[['icustay_id', time_var]],
                      df[['icustay_id','value','label','uom','delta']],
                     left_on= 'icustay_id',
                     right_on= 'icustay_id',
                      how='left') #merging icustay_id with time_var, where value,label,uom, and delta are nan's if no value exists for that icustay

    yn_df['value']= yn_df['value'].fillna(value_fill) #converts na to 0 in above na rows.
    yn_df.loc[yn_df.loc[:,'value']!=value_fill, 'value']= 1 #squashes all other values into a binary 1 = yes
    yn_df['delta']= yn_df['delta'].fillna(delta_fill)
    yn_df['delta']= pd.to_timedelta(yn_df['delta']) #filling in the time delta to time =0 for filled rows
    yn_df['uom']= yn_df['uom'].fillna(uom_fill)
    yn_df.loc[yn_df.loc[:,'uom']!=uom_fill, 'uom']= uom_fill
    yn_df['label']= yn_df['label'].fillna(label_fill)
    
    return(yn_df)

time: 19.8 ms


In [16]:
# date that i ran the below code on the files.

#date= '14112018'

time: 1.08 ms


## vaso dose

In [17]:
#vaso_dose_72.head()
# vaso_dose_72= vaso_dose_72.rename(
#     columns={'starttime':'charttime','label':'vaso_type'})#.drop(labels='endtime', axis=1) 
#renaming starttime to charttime and dropping endtime

dfs['vaso_dose_72']= dfs['vaso_dose_72'].rename(
    columns={'starttime':'charttime','label':'vaso_type'})

time: 24.9 ms


In [18]:
len(dfs['vaso_dose_72'])

26827

time: 11.7 ms


In [19]:
dfs['vaso_dose_72'].head()

,icustay_id,charttime,endtime,vaso_rate,vaso_amount,amount_uom,rate_uom,vaso_type,first_pos_else_neg_ssc,delta
2,200290.0,2102-12-26 03:15:00,2102-12-26 04:00:00,0.250072,1.073559,mg,mcg/kg/min,phenylephrine,2102-12-26 18:54:00,-1 days +08:21:00.000000000
4,200349.0,2139-06-02 16:31:00,2139-06-02 18:10:00,0.012030,0.073958,mg,mcg/kg/min,epinephrine,2139-06-02 19:50:00,-1 days +20:41:00.000000000
6,200349.0,2139-06-01 23:51:00,2139-06-02 09:05:00,0.404244,3.729939,mg,mcg/kg/min,norepinephrine,2139-06-02 19:50:00,-1 days +04:01:00.000000000
9,200952.0,2139-09-25 12:00:00,2139-09-25 21:59:00,0.080068,1.962759,mg,mcg/kg/min,epinephrine,2139-09-25 19:43:00,-1 days +16:17:00.000000000
10,200952.0,2139-09-25 21:59:00,2139-09-26 07:38:00,0.060601,2.000000,mg,mcg/kg/min,epinephrine,2139-09-25 19:43:00,0 days 02:16:00.000000000


time: 16.1 ms


In [20]:
#removing units/hour because that is a different use of vasopressin
dfs['vaso_dose_72'] = dfs['vaso_dose_72'].loc[dfs['vaso_dose_72'].loc[:,'rate_uom']!= 'units/hour',:]
dfs['vaso_dose_72'] = dfs['vaso_dose_72'].loc[dfs['vaso_dose_72'].loc[:,'rate_uom']!= 'Uhr',:]

time: 12.3 ms


In [21]:
len(dfs['vaso_dose_72'])

26827

time: 2.24 ms


In [22]:
dfs['vaso_dose_72']['rate_uom'].unique()

array(['mcg/kg/min', 'mcgkgmin', 'Umin'], dtype=object)

time: 13 ms


#### removing outliers/extreme values

In [23]:
# Use transform to add a column back to the orig df from a groupby aggregation, transform returns a Series with its index aligned to the orig df:
def vaso_outlier_removal(df):
    test_group=(dfs['vaso_dose_72'][['vaso_rate','rate_uom','amount_uom','vaso_type']]#.groupby('vaso_type', as_index=False)
         .groupby(['vaso_type','rate_uom'])
             )

    dfs['vaso_dose_72']['std']=test_group.transform(lambda x : x.std())
    dfs['vaso_dose_72']['mean']=test_group.transform(lambda x : x.mean())

    normal_high_value= pd.DataFrame({
        'vaso_type' : ['dobutamine','dopamine','epinephrine','norepinephrine','vasopressin','phenylephrine'],
        'high_value': [40, 20, 0.5, 1, 0.1, 2] #highest values one might expect to see in a clinic, ie above this is likely erroneous
    }) #found from literature, see notes

    dfs['vaso_dose_72'] = pd.merge(dfs['vaso_dose_72'], normal_high_value, left_on='vaso_type', right_on='vaso_type')

    vaso_dose_72_rmout =(dfs['vaso_dose_72'][
        ~((dfs['vaso_dose_72']['vaso_rate'] > dfs['vaso_dose_72']['high_value']) & ((dfs['vaso_dose_72']['vaso_rate']-dfs['vaso_dose_72']['mean'])>= (3*dfs['vaso_dose_72']['std'])))
        ])
    #ie vaso_dose_72_rmout is a dataframe of all rows that excludes rows where vaso rate > literature high value and where vaso_rate >3sd from teh mean
    return(vaso_dose_72_rmout)

time: 33.5 ms


In [24]:
dfs['vaso_dose_72']= vaso_outlier_removal(dfs['vaso_dose_72'])

time: 87.3 ms


In [25]:
len(dfs['vaso_dose_72']) #26660, down from 26827

26660

time: 1.52 ms


In [26]:
dfs['vaso_dose_72']['vaso_type'].unique()

array(['phenylephrine', 'epinephrine', 'norepinephrine', 'vasopressin',
       'dopamine', 'dobutamine'], dtype=object)

time: 2.97 ms


In [27]:
dfs['vaso_dose_72']['rate_uom'].unique()

array(['mcg/kg/min', 'mcgkgmin', 'Umin'], dtype=object)

time: 2.67 ms


In [28]:
#standardizing names, dropping unneeded columns for analysis
dfs['vaso_dose_72']= dfs['vaso_dose_72'].drop(['vaso_amount', 'amount_uom','std','mean','high_value'], axis=1)
dfs['vaso_dose_72']= dfs['vaso_dose_72'].rename(index=str, columns={'vaso_rate': 'value', 'rate_uom':'uom','vaso_type':'label'})

time: 26.3 ms


In [29]:
dfs['vaso_dose_72']['label'].unique()

array(['phenylephrine', 'epinephrine', 'norepinephrine', 'vasopressin',
       'dopamine', 'dobutamine'], dtype=object)

time: 9.33 ms


In [30]:
dfs['vaso_dose_72'].head()

,icustay_id,charttime,endtime,value,uom,label,first_pos_else_neg_ssc,delta
0,200290.0,2102-12-26 03:15:00,2102-12-26 04:00:00,0.250072,mcg/kg/min,phenylephrine,2102-12-26 18:54:00,-1 days +08:21:00.000000000
1,201906.0,2178-12-26 21:10:00,2178-12-27 01:03:00,4.855967,mcg/kg/min,phenylephrine,2178-12-26 05:16:00,0 days 15:54:00.000000000
2,201906.0,2178-12-27 01:03:00,2178-12-27 03:35:00,4.873294,mcg/kg/min,phenylephrine,2178-12-26 05:16:00,0 days 19:47:00.000000000
3,201906.0,2178-12-27 03:35:00,2178-12-27 05:59:00,4.855967,mcg/kg/min,phenylephrine,2178-12-26 05:16:00,0 days 22:19:00.000000000
4,202639.0,2163-04-24 16:00:00,2163-04-24 17:43:00,3.507000,mcg/kg/min,phenylephrine,2163-04-24 02:38:00,0 days 13:22:00.000000000


time: 25.8 ms


In [31]:
epinephrine_df= dfs['vaso_dose_72'][dfs['vaso_dose_72']['label']=='epinephrine']
norepinephrine_df= dfs['vaso_dose_72'][dfs['vaso_dose_72']['label']=='norepinephrine']
phenylephrine_df= dfs['vaso_dose_72'][dfs['vaso_dose_72']['label']=='phenylephrine']
vasopressin_df= dfs['vaso_dose_72'][dfs['vaso_dose_72']['label']=='vasopressin']
dopamine_df= dfs['vaso_dose_72'][dfs['vaso_dose_72']['label']=='dopamine']
dobutamine_df= dfs['vaso_dose_72'][dfs['vaso_dose_72']['label']=='dobutamine']


time: 30.2 ms


In [32]:
#(could be transfered to a different spdsheet for collapsing values)
#y/n convert, seperating out vaso_dose into 6 constitutient dataframes, and for each am collapsing values down to does pt have in time window y/n? 

epinephrine_df=yn_convert(epinephrine_df, label_fill='epinephrine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var='first_pos_else_neg_ssc')
norepinephrine_df=yn_convert(norepinephrine_df, label_fill='norepinephrine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var='first_pos_else_neg_ssc')
phenylephrine_df=yn_convert(phenylephrine_df, label_fill='phenylephrine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var='first_pos_else_neg_ssc')
vasopressin_df=yn_convert(vasopressin_df, label_fill='vasopressin', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var='first_pos_else_neg_ssc')
dopamine_df=yn_convert(dopamine_df, label_fill='dopamine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var='first_pos_else_neg_ssc')
dobutamine_df=yn_convert(dobutamine_df, label_fill='dobutamine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var='first_pos_else_neg_ssc')


time: 606 ms


In [33]:
save_path

'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/processed/PA_project'

time: 2.29 ms


In [34]:
pd.DataFrame(epinephrine_df).to_csv(Path(
    save_path+ '/%s_epinephrine_prepped.csv' %(date)))
del(epinephrine_df)

pd.DataFrame(norepinephrine_df).to_csv(Path(
    save_path+'/%s_norepinephrine_prepped.csv' %(date)))
del(norepinephrine_df)

pd.DataFrame(phenylephrine_df).to_csv(Path(
    save_path+'/%s_phenylephrine_prepped.csv' %(date)))
del(phenylephrine_df)

pd.DataFrame(vasopressin_df).to_csv(Path(
    save_path+'/%s_vasopressin_prepped.csv' %(date)))
del(vasopressin_df)

pd.DataFrame(dopamine_df).to_csv(Path(
    save_path+'/%s_dopamine_prepped.csv' %(date)))
del(dopamine_df)

pd.DataFrame(dobutamine_df).to_csv(Path(
    save_path+'/%s_dobutamine_prepped.csv' %(date)))
del(dobutamine_df)

del( dfs['vaso_dose_72'])

time: 1.84 s


## vitals -

In [35]:
dfs['vitals_all_nosummary_72'].head()

,subject_id,hadm_id,icustay_id,charttime,valueuom,vitalid,valuenum,first_pos_else_neg_ssc,delta
5,9,150750,220597,2149-11-13 15:20:00,mmHg,MeanArtPress,61.0,2149-11-13 19:30:00,-1 days +19:50:00.000000000
6,9,150750,220597,2149-11-14 11:00:00,mmHg,MeanArtPress,103.0,2149-11-13 19:30:00,0 days 15:30:00.000000000
7,9,150750,220597,2149-11-14 12:00:00,mmHg,MeanArtPress,134.0,2149-11-13 19:30:00,0 days 16:30:00.000000000
12,9,150750,220597,2149-11-13 07:00:00,mmHg,MeanArtPress,85.0,2149-11-13 19:30:00,-1 days +11:30:00.000000000
13,9,150750,220597,2149-11-14 02:00:00,mmHg,MeanArtPress,96.0,2149-11-13 19:30:00,0 days 06:30:00.000000000


time: 20.6 ms


In [36]:
dfs['vitals_all_nosummary_72'] = dfs['vitals_all_nosummary_72'].rename(index=str, columns={"valueuom":"uom","vitalid":'label', 'valuenum':'value'}) #change valueom to uom
dfs['vitals_all_nosummary_72'] = dfs['vitals_all_nosummary_72'].loc[dfs['vitals_all_nosummary_72']['label'].notnull(),:]#.count() #removing null values

dfs['vitals_all_nosummary_72'].loc[dfs['vitals_all_nosummary_72'].loc[:,'uom']=='BPM','uom']='bpm'

#overall the values are extremely similar and are likely the same thing
#i will combine them.
dfs['vitals_all_nosummary_72'].loc[
    (dfs['vitals_all_nosummary_72']['label']=='RespRate') & 
    (dfs['vitals_all_nosummary_72']['uom']=='bpm'),'uom']='insp/min'

dfs['vitals_all_nosummary_72'].loc[
    (dfs['vitals_all_nosummary_72']['label']=='TempC') & 
    (dfs['vitals_all_nosummary_72']['uom']=='?C'),'uom']='Deg. C'

dfs['vitals_all_nosummary_72'].loc[
    (dfs['vitals_all_nosummary_72']['label']=='TempF') & 
    (dfs['vitals_all_nosummary_72']['uom']=='Deg. F'),'uom']='Deg. C'

dfs['vitals_all_nosummary_72'].loc[
    (dfs['vitals_all_nosummary_72']['label']=='TempF') & 
    (dfs['vitals_all_nosummary_72']['uom']=='?F'),'uom']='Deg. C'

dfs['vitals_all_nosummary_72'].loc[
    (dfs['vitals_all_nosummary_72']['label']=='TempF'),'label']='temperature'

dfs['vitals_all_nosummary_72'].loc[
    (dfs['vitals_all_nosummary_72']['label']=='TempC'),'label']='temperature'

time: 6.41 s


- glucose max
- glucose min
- diasBP min
- heartrate min
- meanart pressure min
- RespRate min
- SYSbp min
- TEMPC min

In [37]:
# #writing a function to aggregate for time length

# def timewindow_agg(df, label='', roundwindow= '120min', value_col='value', transform_fxn=max):
#     df['rounded_time']=pd.to_datetime(df['charttime']).dt.round(roundwindow)
#     subset_df=(df.loc[df.loc[:,'label']==label, :])

#     #test_df4.groupby(['icustay_id','rounded_time'])['value'].apply(lambda x: max(x)).sort_values() #works!test_df4
#     #getting the rows for the max value for each rounded_time. 
#     idx= subset_df.groupby(['icustay_id','rounded_time'])['value'].transform(transform_fxn) ==subset_df['value']
#     (df.loc[df.loc[:,'label']==label, :])= subset_df[idx]
#     return(df)

time: 2.29 ms


In [38]:
#erroneous value cutoff summary
## setting a conservative threshold for erroneous values to not skew my data.

(dfs['vitals_all_nosummary_72'].loc[(dfs['vitals_all_nosummary_72'].loc[:,'icustay_id']==228393.0) &
                         (dfs['vitals_all_nosummary_72'].loc[:,'label']=='Glucose') &
                         (dfs['vitals_all_nosummary_72'].loc[:,'value']>99999), 'value'])=np.nan

(dfs['vitals_all_nosummary_72'].loc[(dfs['vitals_all_nosummary_72'].loc[:,'label']=='Glucose') &
                         (dfs['vitals_all_nosummary_72'].loc[:,'value']>99998), 'value'])=np.nan

(dfs['vitals_all_nosummary_72'].loc[(dfs['vitals_all_nosummary_72'].loc[:,'label']=='Glucose') &
                         (dfs['vitals_all_nosummary_72'].loc[:,'value']<15), 'value'])=np.nan

(dfs['vitals_all_nosummary_72'].loc[(dfs['vitals_all_nosummary_72'].loc[:,'label']=='DiasBP') &
                         (dfs['vitals_all_nosummary_72'].loc[:,'value']<15), 'value'])=np.nan

(dfs['vitals_all_nosummary_72'].loc[(dfs['vitals_all_nosummary_72'].loc[:,'label']=='HeartRate') &
                         (dfs['vitals_all_nosummary_72'].loc[:,'value'].between(1,29)), 'value'])=np.nan

(dfs['vitals_all_nosummary_72'].loc[(dfs['vitals_all_nosummary_72'].loc[:,'label']=='RespRate') &
                         (dfs['vitals_all_nosummary_72'].loc[:,'value']<4), 'value'])=np.nan

(dfs['vitals_all_nosummary_72'].loc[(dfs['vitals_all_nosummary_72'].loc[:,'label']=='SysBP') &
                         (dfs['vitals_all_nosummary_72'].loc[:,'value']<40), 'value'])=np.nan

(dfs['vitals_all_nosummary_72'].loc[(dfs['vitals_all_nosummary_72'].loc[:,'label']=='TempC') &
                         (dfs['vitals_all_nosummary_72'].loc[:,'value']<28), 'value'])=np.nan

time: 1.79 s


In [39]:
dfs['vitals_all_nosummary_72'] = dfs['vitals_all_nosummary_72'].loc[dfs['vitals_all_nosummary_72']['value'].notnull(),:]#.count() #6930 NULL values


time: 708 ms


In [40]:
# pd.DataFrame(dfs['vitals_all_nosummary_72']).to_csv(Path(
#     save_path+'/%s_vitals_prepped.csv' %(date)))
# del dfs['vitals_all_nosummary_72']

save_df(dfs['vitals_all_nosummary_72'], 'vitals')
del(dfs['vitals_all_nosummary_72'])

time: 17 s


# labs -

In [41]:
dfs['labs_all_nosummary_72']= dfs['labs_all_nosummary_72'].rename(
    columns={'valuenum':'value'}) #changing valuenum to value


time: 173 ms


In [42]:
dfs['labs_all_nosummary_72'].groupby('label')['uom'].value_counts() #looks good

label        uom    
ALBUMIN      g/dL        6727
ANION GAP    mEq/L      28605
BANDS        %           2956
BICARBONATE  mEq/L      28894
BILIRUBIN    mg/dL      10640
BUN          mg/dL      29249
CHLORIDE     mEq/L      32107
CREATININE   mg/dL      29365
GLUCOSE      mg/dL      36551
HEMATOCRIT   %          35878
HEMOGLOBIN   g/dL       30354
LACTATE      mmol/L     21292
PLATELET     K/uL       27081
POTASSIUM    mEq/L      38885
PT           SECONDS    20108
PTT          sec        20724
SODIUM       mEq/L      33210
WBC          K/uL       26418
Name: uom, dtype: int64

time: 174 ms


In [43]:
#outliers

time: 735 µs


In [44]:
#summary value removal- was explored and coded adhoc, difficult to automate
(dfs['labs_all_nosummary_72'].loc[
    (dfs['labs_all_nosummary_72'].loc[:,'icustay_id']==261887) & 
    (dfs['labs_all_nosummary_72'].loc[:,'label']=='CHLORIDE')
    & (dfs['labs_all_nosummary_72'].loc[:,'value']==3.4),'value'])=np.nan

(dfs['labs_all_nosummary_72'].loc[
    (dfs['labs_all_nosummary_72'].loc[:,'icustay_id']==236290) & 
    (dfs['labs_all_nosummary_72'].loc[:,'label']=='CHLORIDE')
    & (dfs['labs_all_nosummary_72'].loc[:,'value']==11.0),'value'])=np.nan

(dfs['labs_all_nosummary_72'].loc[
    (dfs['labs_all_nosummary_72'].loc[:,'icustay_id']==292769) & 
    (dfs['labs_all_nosummary_72'].loc[:,'label']=='INR')
    & (dfs['labs_all_nosummary_72'].loc[:,'value']==28.1),'value'])=np.nan

(dfs['labs_all_nosummary_72'].loc[
    (dfs['labs_all_nosummary_72'].loc[:,'icustay_id']==298457) & 
    (dfs['labs_all_nosummary_72'].loc[:,'label']=='INR')
    & (dfs['labs_all_nosummary_72'].loc[:,'value']==48.8),'value'])=np.nan

(dfs['labs_all_nosummary_72'].loc[
    (dfs['labs_all_nosummary_72'].loc[:,'icustay_id']==234174) & 
    (dfs['labs_all_nosummary_72'].loc[:,'label']=='INR')
    & (dfs['labs_all_nosummary_72'].loc[:,'value']==48.7),'value'])=np.nan

(dfs['labs_all_nosummary_72'].loc[
    (dfs['labs_all_nosummary_72'].loc[:,'icustay_id']==290264) & 
    (dfs['labs_all_nosummary_72'].loc[:,'label']=='INR')
    & (dfs['labs_all_nosummary_72'].loc[:,'value']==42.0),'value'])=np.nan

(dfs['labs_all_nosummary_72'].loc[
    (dfs['labs_all_nosummary_72'].loc[:,'icustay_id']==290264) & 
    (dfs['labs_all_nosummary_72'].loc[:,'label']=='INR')
    & (dfs['labs_all_nosummary_72'].loc[:,'value']==22.8),'value'])=np.nan

time: 413 ms


In [45]:
dfs['labs_all_nosummary_72']= dfs['labs_all_nosummary_72'].loc[dfs['labs_all_nosummary_72'].loc[:,'value'].notnull(),:] #removing null values


time: 50.1 ms


In [46]:
dfs['labs_all_nosummary_72'].head()

,subject_id,hadm_id,icustay_id,charttime,label,value,uom,first_pos_else_neg_ssc,delta
0,9,150750,220597,2149-11-14 07:47:00,SODIUM,137.0,mEq/L,2149-11-13 19:30:00,0 days 12:17:00.000000000
6,9,150750,220597,2149-11-13 03:20:00,SODIUM,130.0,mEq/L,2149-11-13 19:30:00,-1 days +07:50:00.000000000
7,9,150750,220597,2149-11-13 16:04:00,SODIUM,129.0,mEq/L,2149-11-13 19:30:00,-1 days +20:34:00.000000000
8,9,150750,220597,2149-11-13 19:30:00,SODIUM,131.0,mEq/L,2149-11-13 19:30:00,0 days 00:00:00.000000000
9,9,150750,220597,2149-11-14 01:48:00,SODIUM,133.0,mEq/L,2149-11-13 19:30:00,0 days 06:18:00.000000000


time: 27.1 ms


In [47]:
#removing unwanted values
unwanted_values= ['HEMATOCRIT','ANION GAP','PT','ALBUMIN']
dfs['labs_all_nosummary_72']= dfs['labs_all_nosummary_72'].loc[~dfs['labs_all_nosummary_72'].loc[:,'label'].isin(unwanted_values),:]

time: 58.6 ms


In [48]:
pd.DataFrame(dfs['labs_all_nosummary_72']).to_csv(Path(
    save_path+'/%s_labs_prepped.csv' %(date)))
del(dfs['labs_all_nosummary_72'])

time: 2.79 s


## vent category -

In [51]:
dfs['ventcategory_df']['uom']='mech/O2/none category'
dfs['ventcategory_df'].head()

,icustay_id,day,value,uom,first_pos_else_neg_ssc
0,200014.0,-1,Mech,mech/O2/none category,2105-02-17 00:00:00
1,200014.0,0,Mech,mech/O2/none category,2105-02-17 00:00:00
2,200019.0,-1,Mech,mech/O2/none category,2178-07-08 21:51:00
3,200019.0,0,Mech,mech/O2/none category,2178-07-08 21:51:00
4,200030.0,-1,Oxygen,mech/O2/none category,2150-11-15 02:08:00


time: 8.51 ms


In [54]:
dfs['ventcategory_df']=dfs['ventcategory_df'].rename(index=str, columns={'day':'delta'})
dfs['ventcategory_df']['label']='vent_recieved'
dfs['ventcategory_df']['day']=pd.to_timedelta(dfs['ventcategory_df']['day'], unit='d')
#dfs['ventcategory_df']= dfs['ventcategory_df'].drop(columns=['day'], axis=1) #removing day column

time: 12.9 ms


In [ ]:
# #melting value columns into long format
# ventcategory_df= ventcategory_df.melt(
#     id_vars=['icustay_id','uom','t_0'],
#     var_name='label',
#     value_name='value')

# #adding a delta column
# ventcategory_df['delta']= np.nan
# ventcategory_df.loc[ventcategory_df['label']=='vent_day1','delta']= pd.to_timedelta(0,'days')
# ventcategory_df.loc[ventcategory_df['label']=='vent_day2','delta']= pd.to_timedelta(1,'days')
# ventcategory_df.loc[ventcategory_df['label']=='vent_day3','delta']= pd.to_timedelta(2,'days')

In [ ]:
#collapsing into 1 column for 3 days

In [63]:
def vent_day_collapser(x):
    if 'Mech' in list(x.unique()):
        x= 'Mech'
    elif 'Oxygen' in list(x.unique()):
        x= 'Oxygen'
    else:
        x='None'
        
    return(x)


time: 4.1 ms


In [64]:
#collapsing all days into the worst day. (could be transfered to a different spdsheet)
ventcategory_1day_df= dfs['ventcategory_df']
ventcategory_1day_df['value']=ventcategory_1day_df.groupby('icustay_id',as_index=False)['value'].transform(vent_day_collapser)
ventcategory_1day_df= ventcategory_1day_df.drop_duplicates(['icustay_id','value']).sort_values('icustay_id') #
ventcategory_1day_df= ventcategory_1day_df.loc[ventcategory_1day_df.loc[:,'icustay_id'].isin(icustay_id),:] #had icustay ids not in final cohort, fail safe mesure
#ventcategory_1day_df['label']= 'vent_recieved'

time: 8.98 s


In [65]:
ventcategory_1day_df.head()

,icustay_id,delta,value,uom,first_pos_else_neg_ssc,label
0,200014.0,-1 days,Mech,mech/O2/none category,2105-02-17 00:00:00,vent_recieved
2,200019.0,-1 days,Mech,mech/O2/none category,2178-07-08 21:51:00,vent_recieved
4,200030.0,-1 days,Oxygen,mech/O2/none category,2150-11-15 02:08:00,vent_recieved
6,200033.0,-1 days,Mech,mech/O2/none category,2198-08-11 00:31:00,vent_recieved
8,200055.0,0 days,None,mech/O2/none category,2179-03-17 12:15:00,vent_recieved


time: 8.9 ms


In [66]:
pd.DataFrame(ventcategory_1day_df).to_csv(Path(
    save_path+'/%s_ventcategory_prepped.csv' %(date)))
del(ventcategory_1day_df,dfs['ventcategory_df'] )

time: 151 ms


## weight and height firstday -
i explored weightdurations and it had more missing values than weightfirstday, so i will use that. we can revisit this if we need longitudinal weights

In [67]:
dfs['weightfirstday_df']['uom']='kg'

time: 1.19 ms


In [68]:
dfs['weightfirstday_df'].head()  
#weight column is the conglomerate of weight_admin>weight_daily> weight_echoinhosp> weight_echoprehosp

,icustay_id,weight,weight_admit,weight_daily,weight_echoinhosp,weight_echoprehosp,uom
8,200014,62.0,62.000000,NaN,NaN,NaN,kg
10,200019,65.0,65.000000,NaN,NaN,NaN,kg
16,200030,113.6,113.599998,113.599998,NaN,NaN,kg
18,200033,74.0,74.000000,NaN,NaN,NaN,kg
33,200055,56.0,56.000000,NaN,NaN,NaN,kg


time: 9.46 ms


In [69]:
dfs['weightfirstday_df']= dfs['weightfirstday_df'][dfs['weightfirstday_df']['weight'].notnull()]
dfs['weightfirstday_df']= dfs['weightfirstday_df'][['icustay_id','weight','uom']]

time: 3.84 ms


In [70]:
dfs['weightfirstday_df']['label']= 'weight'
dfs['weightfirstday_df']=dfs['weightfirstday_df'].rename(index=str, columns={'weight':'value'})

#adding the assumed first day delta column to standardize all columns
dfs['weightfirstday_df']['delta']=pd.to_timedelta(0,'days')

#adding t_0
dfs['weightfirstday_df']= pd.merge(dfs['weightfirstday_df'], final_pt_df2[['icustay_id',time_var]], left_on='icustay_id', right_on='icustay_id')

time: 63.8 ms


In [71]:
pd.DataFrame(dfs['weightfirstday_df']).to_csv(Path(
    save_path+'/%s_weight_prepped.csv' %(date)))
del(dfs['weightfirstday_df'])

time: 159 ms


In [72]:
dfs['heightfirstday_df']['uom']='cm'

time: 1.22 ms


In [73]:
dfs['heightfirstday_df']= dfs['heightfirstday_df'][dfs['heightfirstday_df']['height'].notnull()]
dfs['heightfirstday_df']= dfs['heightfirstday_df'][['icustay_id','height','uom']]

time: 3.31 ms


In [74]:
dfs['heightfirstday_df']['label']= 'height'
dfs['heightfirstday_df']=dfs['heightfirstday_df'].rename(index=str, columns={'height':'value'})

#adding the assumed first day delta column to standardize all columns
dfs['heightfirstday_df']['delta']=pd.to_timedelta(0,'days')
#adding t_0
dfs['heightfirstday_df']= pd.merge(dfs['heightfirstday_df'], final_pt_df2[['icustay_id',time_var]], left_on='icustay_id', right_on='icustay_id')

time: 67.1 ms


In [75]:
pd.DataFrame(dfs['heightfirstday_df']).to_csv(Path(
    save_path+'/%s_height_prepped.csv' %(date)))
del(dfs['heightfirstday_df'])

time: 120 ms


# UTI

In [76]:
dfs['uti_all_72']['value'].unique()#seems good #all uti within clinical timewindow

array(['NEG', 'SM', 'MOD', 'TR', 'POS', 'LG',
       'COMPUTER NETWORK FAILURE. TEST NOT RESULTED.', 'SMALL'],
      dtype=object)

time: 13.1 ms


In [77]:
dfs['uti_all_72']= dfs['uti_all_72'].loc[(dfs['uti_all_72'].loc[:,'value']!='NEG')&
               dfs['uti_all_72'].loc[:,'value'].notnull(),:] #filter to only pos rows

dfs['uti_all_72']= dfs['uti_all_72'].loc[(dfs['uti_all_72'].loc[:,'value']!='COMPUTER NETWORK FAILURE. TEST NOT RESULTED.')&
               dfs['uti_all_72'].loc[:,'value'].notnull(),:] #filter to only pos rows
dfs['uti_all_72']= dfs['uti_all_72'].drop_duplicates(subset=['hadm_id','value','charttime'])

dfs['uti_all_72'].loc[dfs['uti_all_72'].loc[:,'value'].notnull(),'value']= 1
dfs['uti_all_72'].loc[dfs['uti_all_72'].loc[:,'value'].isna(),'value']= 0 

time: 108 ms


In [78]:
dfs['uti_all_72'].head()

,subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom,label,fluid,category,loinc_code,icd9_code,first_pos_else_neg_ssc,delta
255,186,164405.0,51486,2176-09-27 20:30:00,1,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,4560,2176-09-27 22:24:00,-1 days +22:06:00.000000000
338,191,136614.0,51486,2196-04-11 17:27:00,1,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,51881,2196-04-12 00:00:00,-1 days +17:27:00.000000000
353,191,136614.0,51486,2196-04-12 19:15:00,1,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,51881,2196-04-12 00:00:00,0 days 19:15:00.000000000
911,279,192224.0,51486,2164-06-14 18:00:00,1,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,1561,2164-06-15 07:00:00,-1 days +11:00:00.000000000
974,339,107660.0,51486,2188-04-12 20:23:00,1,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,5772,2188-04-13 15:22:00,-1 days +05:01:00.000000000


time: 13 ms


In [79]:
# ###coding any positive leukocyte or nitrate between day 1,2 or 3 for all patients. if time granularity is later needed for models, need to revisit this data to get the exact delta value

# def uti_categorization(final_pt_df2):
    
#     nit_df_3dayannotation= final_pt_df2[['subject_id','hadm_id','t_0']]
#     nit_df_3dayannotation['uti_day1']='Neg/Not_tested'
#     nit_df_3dayannotation['uti_day2']='Neg/Not_tested' 
#     nit_df_3dayannotation['uti_day3']='Neg/Not_tested' 
    
#     leuk_df_3dayannotation= final_pt_df2[['subject_id','hadm_id','t_0']]
#     leuk_df_3dayannotation['uti_day1']='Neg/Not_tested'
#     leuk_df_3dayannotation['uti_day2']='Neg/Not_tested' 
#     leuk_df_3dayannotation['uti_day3']='Neg/Not_tested' 

#     leuk_pos_1= uti_leuk_pos.loc[uti_leuk_pos['delta'].between('0 days','1 days'),'hadm_id'].tolist()
#     leuk_pos_2= uti_leuk_pos.loc[uti_leuk_pos['delta'].between('1 days','2 days'),'hadm_id'].tolist()
#     leuk_pos_3= uti_leuk_pos.loc[uti_leuk_pos['delta'].between('2 days','3 days'),'hadm_id'].tolist()
    
#     nit_pos_1= uti_nit_pos.loc[uti_nit_pos['delta'].between('0 days','1 days'),'hadm_id'].tolist()
#     nit_pos_2= uti_nit_pos.loc[uti_nit_pos['delta'].between('1 days','2 days'),'hadm_id'].tolist()
#     nit_pos_3= uti_nit_pos.loc[uti_nit_pos['delta'].between('2 days','3 days'),'hadm_id'].tolist()
    
# #     both_pos_1= list(set(leuk_pos_1) & set(nit_pos_1))
# #     both_pos_2= list(set(leuk_pos_2) & set(nit_pos_2))
# #     both_pos_3= list(set(leuk_pos_3) & set(nit_pos_3))

#     leuk_df_3dayannotation.loc[leuk_df_3dayannotation.loc[:,'hadm_id'].isin(leuk_pos_1),'uti_day1']='pos'
#     leuk_df_3dayannotation.loc[leuk_df_3dayannotation.loc[:,'hadm_id'].isin(leuk_pos_2),'uti_day2']='pos'
#     leuk_df_3dayannotation.loc[leuk_df_3dayannotation.loc[:,'hadm_id'].isin(leuk_pos_3),'uti_day3']='pos'
    
#     nit_df_3dayannotation.loc[nit_df_3dayannotation.loc[:,'hadm_id'].isin(nit_pos_1),'uti_day1']='pos'
#     nit_df_3dayannotation.loc[nit_df_3dayannotation.loc[:,'hadm_id'].isin(nit_pos_2),'uti_day2']='pos'
#     nit_df_3dayannotation.loc[nit_df_3dayannotation.loc[:,'hadm_id'].isin(nit_pos_3),'uti_day3']='pos'
    
# #     uti_df_3dayannotation.loc[uti_df_3dayannotation.loc[:,'hadm_id'].isin(both_pos_1),'uti_day1']='both_pos'
# #     uti_df_3dayannotation.loc[uti_df_3dayannotation.loc[:,'hadm_id'].isin(both_pos_2),'uti_day2']='both_pos'
# #     uti_df_3dayannotation.loc[uti_df_3dayannotation.loc[:,'hadm_id'].isin(both_pos_3),'uti_day3']='both_pos'
    
#     return(leuk_df_3dayannotation, nit_df_3dayannotation)

time: 8.25 ms


In [80]:
#leuk_categorization, nit_categorization= uti_categorization(final_pt_df2)

time: 788 µs


In [81]:

def uti_categorizer(uti_df):
    "useful to get all rows of days with positive values for patients (if multiple pos in a day there will be only 1 row for that day). ie more longitudinal format "
    
    #gives the max pos or neg value per day for 
    df_timewindow_perday=uti_df.groupby(['hadm_id','delta'], as_index=False)['value'].agg({'value':'max'})  
    df_timewindow_perday= pd.merge(df_timewindow_perday,final_pt_df2[['icustay_id','hadm_id', time_var]], left_on='hadm_id', right_on='hadm_id', how='left')
    df_timewindow_perday=df_timewindow_perday.sort_values(['hadm_id','value','delta'], ascending=[True,False,True])
    return(df_timewindow_perday)

def uti_collapser(uti_df, label):
    "collapsing longitudinal data into 1 value. will return pos or neg if patient has a positive uti in their stay. one row per icustay_id"
    
    df_timewindow_perday=uti_df.groupby(['hadm_id','delta'], as_index=False)['value'].agg({'value':'max'})
    first_pos= df_timewindow_perday.drop_duplicates(['hadm_id'])
    collapsed= pd.merge(final_pt_df2[['hadm_id','icustay_id','subject_id', time_var]],first_pos, left_on='hadm_id', right_on='hadm_id', how='left')
    
    collapsed['value']= collapsed['value'].fillna(0)
    collapsed.loc[collapsed.loc[:,'value']==1,'value']= 'pos'
    collapsed.loc[collapsed.loc[:,'value']==0,'value']= 'Neg/Not_tested'
    
    collapsed['delta']= collapsed['delta'].fillna(pd.Timedelta(1, unit='d'))
    
    collapsed['label']= label
    
    collapsed['uom']='pos/neg category'
    return(collapsed)

time: 55.5 ms


In [82]:
uti_nit_pos= dfs['uti_all_72'][dfs['uti_all_72']['label']=="Nitrite"]
uti_leuk_pos= dfs['uti_all_72'][dfs['uti_all_72']['label']=="Leukocytes"]

leuk_collapsed= uti_collapser(uti_leuk_pos, 'leukocyte')
nit_collapsed= uti_collapser(uti_nit_pos, 'nitrite')


time: 57.7 ms


In [83]:
save_df(leuk_collapsed, 'leuk')
del(leuk_collapsed)

save_df(nit_collapsed, 'nit')
del(nit_collapsed)
del(dfs['uti_all_72'])

time: 225 ms


# bloodgas

In [84]:

dfs['bg_all_nosummary_72']= dfs['bg_all_nosummary_72'].loc[dfs['bg_all_nosummary_72']['value'].notnull(),:]
dfs['bg_all_nosummary_72'] = dfs['bg_all_nosummary_72'].rename(index=str, columns={'valueuom':'uom'})
dfs['bg_all_nosummary_72']= dfs['bg_all_nosummary_72'].loc[~(dfs['bg_all_nosummary_72'].loc[:,'value']=='.'),:]

#may need to remove outliers, haven't done as of 10/22/18
dfs['bg_all_nosummary_72'].head()

,subject_id,hadm_id,icustay_id,charttime,label,valuenum,value,uom,unique_var,first_pos_else_neg_ssc,delta
82,9,150750,220597,2149-11-13 03:34:00,BASEEXCESS,NaN,-1,mEq/L,2205972149-11-13 03:34:00,2149-11-13 19:30:00,-1 days +08:04:00.000000000
83,9,150750,220597,2149-11-13 03:34:00,CALCIUM,1.05,1.05,mmol/L,2205972149-11-13 03:34:00,2149-11-13 19:30:00,-1 days +08:04:00.000000000
84,9,150750,220597,2149-11-13 03:34:00,LACTATE,1.80,1.8,mmol/L,2205972149-11-13 03:34:00,2149-11-13 19:30:00,-1 days +08:04:00.000000000
85,9,150750,220597,2149-11-13 03:34:00,PCO2,34.00,34,mm Hg,2205972149-11-13 03:34:00,2149-11-13 19:30:00,-1 days +08:04:00.000000000
86,9,150750,220597,2149-11-13 03:34:00,PH,7.42,7.42,units,2205972149-11-13 03:34:00,2149-11-13 19:30:00,-1 days +08:04:00.000000000


time: 546 ms


#### erroneous value removal

NOTE: these are hard coded to positioning. if anything in the bg_all_nosummary is changed than these positions need to be updated. 

In [85]:
##calcium
#fixing the calcium errors w/o hard coding
(dfs['bg_all_nosummary_72'].loc[
    (dfs['bg_all_nosummary_72'].loc[:,'icustay_id']==249571) & 
    (dfs['bg_all_nosummary_72'].loc[:,'label']=='CALCIUM')
    & (dfs['bg_all_nosummary_72'].loc[:,'valuenum']==94.00),'valuenum'])=0.94#.where('valuenum'==94.00))

(dfs['bg_all_nosummary_72'].loc[
    (dfs['bg_all_nosummary_72'].loc[:,'icustay_id']==249571) & 
    (dfs['bg_all_nosummary_72'].loc[:,'label']=='CALCIUM')
    & (dfs['bg_all_nosummary_72'].loc[:,'value']=='094'),'value'])=0.94#.where('valuenum'==94.00))


(dfs['bg_all_nosummary_72'].loc[
    (dfs['bg_all_nosummary_72'].loc[:,'icustay_id']==219600) & 
    (dfs['bg_all_nosummary_72'].loc[:,'label']=='CALCIUM')
    & (dfs['bg_all_nosummary_72'].loc[:,'valuenum']==97.00),'valuenum'])=0.97#.where('valuenum'==94.00))

(dfs['bg_all_nosummary_72'].loc[
    (dfs['bg_all_nosummary_72'].loc[:,'icustay_id']==219600) & 
    (dfs['bg_all_nosummary_72'].loc[:,'label']=='CALCIUM')
    & (dfs['bg_all_nosummary_72'].loc[:,'value']=='097'),'value'])=0.97#.where('valuenum'==94.00))


time: 383 ms


In [86]:
##min chloride
#converting it to a null value without hard coding
(dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'icustay_id']==261887.0) &
                         (dfs['bg_all_nosummary_72'].loc[:,'label']=='CHLORIDE') &
                         (dfs['bg_all_nosummary_72'].loc[:,'valuenum']==3.4),'value'])=np.nan
#converting it to a null value without hard coding
(dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'icustay_id']==261887.0) &
                         (dfs['bg_all_nosummary_72'].loc[:,'label']=='CHLORIDE') &
                         (dfs['bg_all_nosummary_72'].loc[:,'valuenum']==3.4),'valuenum'])=np.nan

#changing the values without hard coding. 
(dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'icustay_id']==236290.0) &
                         (dfs['bg_all_nosummary_72'].loc[:,'label']=='CHLORIDE') &
                         (dfs['bg_all_nosummary_72'].loc[:,'valuenum']==11.0),'valuenum'])=np.nan

(dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'icustay_id']==236290.0) &
                         (dfs['bg_all_nosummary_72'].loc[:,'label']=='CHLORIDE') &
                         (dfs['bg_all_nosummary_72'].loc[:,'valuenum']==11.0),'value'])=np.nan

time: 268 ms


In [87]:
#peep changes summary: 
dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'label']=='PEEP')&
                        (dfs['bg_all_nosummary_72']['valuenum']>38),'valuenum']=np.nan #remove this or set to 50?

time: 65.7 ms


In [88]:
#temp changes summary:

dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'label']=='TEMPERATURE')&
                        (dfs['bg_all_nosummary_72']['icustay_id']==253821)&
                        (dfs['bg_all_nosummary_72']['valuenum']==18.9),
                        'value']= np.nan

dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'label']=='TEMPERATURE')&
                        (dfs['bg_all_nosummary_72']['icustay_id']==253821)&
                        (dfs['bg_all_nosummary_72']['valuenum']==18.9),
                        'valuenum']= np.nan

dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'label']=='TEMPERATURE')&
                        (dfs['bg_all_nosummary_72']['icustay_id']==251788)&
                        (dfs['bg_all_nosummary_72']['valuenum']==10.0),
                        'value']= np.nan

dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'label']=='TEMPERATURE')&
                        (dfs['bg_all_nosummary_72']['icustay_id']==251788)&
                        (dfs['bg_all_nosummary_72']['valuenum']==10.0),
                        'valuenum']= np.nan

time: 244 ms


In [89]:
#fio2 changes summary:

##converting a few values to null, thus removing them from the dataset
dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'label']=='FIO2') &
                        (dfs['bg_all_nosummary_72'].loc[:,'value']=='0'),'value']=np.nan

dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'label']=='FIO2') &
                        (dfs['bg_all_nosummary_72'].loc[:,'value']=='-'),'value']=np.nan


##removing all fio2 values between 1-20.9

dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'label'].isin(['FIO2']))&
                        (dfs['bg_all_nosummary_72']['valuenum'].between(1.0,20.9)),'valuenum']=np.nan


##values between 0-1 were found to be ratios, not %, so converting these to % 

dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'label'].isin(['FIO2']))&
                        (dfs['bg_all_nosummary_72']['valuenum'].between(1.0,20.9)),'valuenum']=np.nan

fio2_dec= dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'label'].isin(['FIO2']))&
                        (dfs['bg_all_nosummary_72']['valuenum'].between(0.0,1.0)),'valuenum']

dfs['bg_all_nosummary_72'].loc[(dfs['bg_all_nosummary_72'].loc[:,'label'].isin(['FIO2']))&
                        (dfs['bg_all_nosummary_72']['valuenum'].between(0.0,1.0)),'valuenum'] = fio2_dec *100

del(fio2_dec)

time: 317 ms


#### removing null values annotated abov

In [90]:
dfs['bg_all_nosummary_72']= dfs['bg_all_nosummary_72'].loc[dfs['bg_all_nosummary_72']['value'].notnull(),:]

time: 50.9 ms


### splitting specimen out for vent vs non-vent bg data

In [91]:
#adding specimen tag to filter only arterial samples for vent data. 
specimen_df= dfs['bg_all_nosummary_72'].loc[dfs['bg_all_nosummary_72'].loc[:,'label']=='SPECIMEN',['unique_var','label','value']]#unique()
specimen_df=specimen_df.rename(index=str, columns={'value':'specimen'})
specimen_df=specimen_df.loc[specimen_df.loc[:,"specimen"]=='ART',:]

dfs['bg_all_nosummary_72']= pd.merge(dfs['bg_all_nosummary_72'],specimen_df[['unique_var','specimen']], left_on='unique_var', right_on='unique_var', how='left')
bg_ART_nosummary_72=dfs['bg_all_nosummary_72'].loc[dfs['bg_all_nosummary_72'].loc[:,'specimen']=='ART',:]
del(specimen_df)

time: 456 ms


In [92]:
bg_labels=['PH','LACTATE','CALCIUM','TEMPERATURE','POTASSIUM',
           'GLUCOSE','HEMOGLOBIN','SODIUM','CHLORIDE','BICARBONATE']
bg_vent_labels=['PCO2','PaO2','PO2','FIO2','PEEP','O2FLOW']

time: 1.25 ms


In [93]:
#restricting to tests that were chosen to be analysed based on %missingness and clinical relevance
dfs['bg_all_nosummary_72']= dfs['bg_all_nosummary_72'].loc[dfs['bg_all_nosummary_72'].loc[:,'label'].isin(bg_labels),:]
bg_ART_nosummary_72= bg_ART_nosummary_72.loc[bg_ART_nosummary_72.loc[:,'label'].isin(bg_vent_labels),:]


time: 56.1 ms


In [94]:
#value = valuenum for all specified bg tests. removing unneeded cols

bg_col=['subject_id','hadm_id','icustay_id','charttime','delta',time_var,'label','valuenum','uom']
bg_ART_nosummary_72= bg_ART_nosummary_72[bg_col]
dfs['bg_all_nosummary_72']= dfs['bg_all_nosummary_72'][bg_col]
del(bg_col)

time: 11.5 ms


In [95]:
bg_ART_nosummary_72['label'].unique()

array(['PCO2', 'PaO2', 'FIO2', 'PEEP', 'O2FLOW'], dtype=object)

time: 4.75 ms


In [96]:
#converting O2 to y/n
o2_flow_df= bg_ART_nosummary_72.loc[bg_ART_nosummary_72.loc[:,'label']=='O2FLOW',:]
o2_flow_df=o2_flow_df.rename(index=str, columns={'valuenum':'value'})
o2_flow_df= yn_convert(o2_flow_df, label_fill='o2_flow', time_var=time_var)

#removing o2_flow from bg_ART
bg_ART_nosummary_72= bg_ART_nosummary_72.loc[bg_ART_nosummary_72.loc[:,'label']!='O2FLOW',:]

time: 88.1 ms


In [97]:
bg_ART_nosummary_72= bg_ART_nosummary_72.loc[bg_ART_nosummary_72.loc[:,'label']!='PEEP',:]

time: 12.1 ms


The PaO2/FiO2 ratio should be calculated using values that are recorded either at the same time stamp, or when the FIO2 occurs within 2 hours BEFORE the PaO2, not the other way around.
For those with no FiO2 recorded within that time frame, you can assume it’s 0.21 (=21%). The worse ratio will be the lowest PaO2 recorded with FiO2 of 1.0 (=100%). So if that is 12, then it will be 12/1 = 12.

In [98]:
#need to groupby icustay_id and charttime for both. pan

pao2= bg_ART_nosummary_72.loc[bg_ART_nosummary_72.loc[:,'label']=='PaO2',:] #n=41799 non-null
fio2= bg_ART_nosummary_72.loc[bg_ART_nosummary_72.loc[:,'label']=='FIO2',:] #n=12311 non-null
pao2['delta']=pd.to_timedelta(pao2['delta'])
fio2['delta']=pd.to_timedelta(fio2['delta'])
#fio2['offset']=fio2['delta'] + pd.to_timedelta(2, unit='h')
pao2['offset']=pao2['delta'] - pd.to_timedelta(2, unit='h') #pao2 should have the offset, not fio2
#pafi_merge=pd.merge(pao2, fio2[['icustay_id','delta','valuenum','uom']], on=['icustay_id','delta'], how='left')

#pafi_merge.loc[pafi_merge.loc[:,'valuenum_y'].notnull(),:] #12310 fio2 values merged over. this is great news, all but 1 have an associated PAO2.
#now need to find teh pao2 values without exact match and find the fio2 within 2 hr.
#if the row is within 2 hours of another row, merge.


time: 230 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [99]:
#removing pao2 and fio2 from bg_ART_nosummary_72
bg_ART_nosummary_72= bg_ART_nosummary_72.loc[bg_ART_nosummary_72.loc[:,'label']!='PaO2',:]
bg_ART_nosummary_72= bg_ART_nosummary_72.loc[bg_ART_nosummary_72.loc[:,'label']!='FIO2',:]


time: 16.6 ms


In [100]:
import pandasql as ps

sqlcode = '''
SELECT A.*, B.valuenum as valuenum2, B.label as label2, B.charttime as charttime2
FROM pao2 as A --grabs from both merged and non-merged as is, or jsut from notmerged if chagned to pao2_notmerged
LEFT JOIN fio2 AS B on A.icustay_id = B.icustay_id
WHERE B.delta BETWEEN A.offset and A.delta;
'''

newdf = ps.sqldf(sqlcode,locals())

#(test_rows.delta >= fio2.delta and test_rows.delta<= fio2.offset)


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandasql/sqldf.py:121: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  index=not any(name is None for name in df.index.names))  # load index into db if all levels are named
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandasql/sqldf.py:121: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  index=not any(name is None for name in df.index.names))  # load index into db if all levels are named


time: 541 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandasql/sqldf.py:121: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  index=not any(name is None for name in df.index.names))  # load index into db if all levels are named


In [101]:
#newdf #15196
#1040 rows from notmerged. 
#grabes from both
newdf['delta']=pd.to_timedelta(newdf['delta'])
newdf['offset']=pd.to_timedelta(newdf['delta'])
ratio_df= pd.merge(pao2, newdf[['icustay_id','delta','valuenum2', 'charttime2']], on=['icustay_id','delta'], how='left')
del(newdf, pao2,fio2)


time: 20.7 ms


In [102]:
#i merged in all rows that matched the 2hr criteria
## For those with no FiO2 recorded within that time frame, you can assume it’s 0.21 (=21%). 
ratio_df.loc[ratio_df.loc[:,'valuenum2'].isnull(),'valuenum2']= 21
ratio_df['ratio']= ratio_df['valuenum']/(ratio_df['valuenum2']/100)
ratio_df.loc[ratio_df['charttime2'].isnull(),'charttime2']= ratio_df.loc[ratio_df['charttime2'].isnull(),'charttime']
ratio_df['deltadelta']=pd.to_datetime(ratio_df['charttime'])-pd.to_datetime(ratio_df['charttime2'])


time: 50.9 ms


In [103]:
ratio_df['ratio'].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.99])


count    21064.000000
mean       471.736686
std        361.486826
min         22.000000
10%        132.675325
25%        240.000000
50%        395.238095
75%        576.190476
99%       1958.904762
max       2785.714286
Name: ratio, dtype: float64

time: 9.18 ms


In [104]:
ratio_df['valuenum'].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.99])


count    21064.000000
mean       130.914071
std         77.073531
min         11.000000
10%         67.000000
25%         83.000000
50%        109.000000
75%        151.000000
99%        443.000000
max        596.000000
Name: valuenum, dtype: float64

time: 10.7 ms


In [105]:
#test_df.sort_values(['icustay_id','delta','deltadelta']).drop_duplicates(['icustay_id','delta','ratio'])

time: 896 µs


In [106]:
### all of the duplicate PaO2 with different FIo2. if a pao2 value has 2 fio2 values that match the 2 hour window, then the one closest time to the fio2 value is chosen. 
ratio_df=ratio_df.sort_values(['icustay_id','delta','deltadelta']).drop_duplicates(['icustay_id','delta','ratio'])
#43715 ->42455

time: 20.4 ms


In [107]:
ratio_df['ratio'].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.99])

count    20359.000000
mean       481.392102
std        363.193621
min         22.000000
10%        139.000000
25%        252.380952
50%        404.000000
75%        585.714286
99%       1971.428571
max       2785.714286
Name: ratio, dtype: float64

time: 28.3 ms


In [108]:
ratio_df['valuenum'].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.99])


count    20359.000000
mean       131.209146
std         77.223885
min         11.000000
10%         67.000000
25%         83.000000
50%        109.000000
75%        151.000000
99%        443.000000
max        596.000000
Name: valuenum, dtype: float64

time: 17.7 ms


In [109]:
#finding worst value:

time: 1.51 ms


In [110]:
ratio_df[ratio_df['icustay_id']==200014]

,subject_id,hadm_id,icustay_id,charttime,delta,first_pos_else_neg_ssc,label,valuenum,uom,offset,valuenum2,charttime2,ratio,deltadelta
3941,9514,127229,200014,2105-02-17 01:08:00,01:08:00,2105-02-17 00:00:00,PaO2,179.0,mm Hg,-1 days +23:08:00,21.0,2105-02-17 01:08:00,852.380952,00:00:00
3942,9514,127229,200014,2105-02-17 04:35:00,04:35:00,2105-02-17 00:00:00,PaO2,183.0,mm Hg,02:35:00,21.0,2105-02-17 04:35:00,871.428571,00:00:00
3943,9514,127229,200014,2105-02-17 11:18:00,11:18:00,2105-02-17 00:00:00,PaO2,142.0,mm Hg,09:18:00,21.0,2105-02-17 11:18:00,676.190476,00:00:00
3944,9514,127229,200014,2105-02-17 15:28:00,15:28:00,2105-02-17 00:00:00,PaO2,155.0,mm Hg,13:28:00,40.0,2105-02-17 15:28:00,387.500000,00:00:00
3946,9514,127229,200014,2105-02-17 17:11:00,17:11:00,2105-02-17 00:00:00,PaO2,96.0,mm Hg,15:11:00,28.0,2105-02-17 17:11:00,342.857143,00:00:00
3945,9514,127229,200014,2105-02-17 17:11:00,17:11:00,2105-02-17 00:00:00,PaO2,96.0,mm Hg,15:11:00,40.0,2105-02-17 15:28:00,240.000000,01:43:00
3947,9514,127229,200014,2105-02-17 20:53:00,20:53:00,2105-02-17 00:00:00,PaO2,95.0,mm Hg,18:53:00,21.0,2105-02-17 20:53:00,452.380952,00:00:00


time: 17.7 ms


In [111]:
# ratio_df.groupby(['icustay_id'], as_index=False)['ratio'].min() #is this what i want for my pao2/fio2 ratio?


f = {'ratio': 'min','delta':'first'}
ratio_min_df=ratio_df.sort_values(['icustay_id','ratio'],ascending=True).groupby('icustay_id', as_index=False).agg(f)

##it's likely that this variable will not explain any additional variation in y than the ventilation category variable we already have. 
##maybe we can stratify patients with a few categories:

##not on vent
##being on vent with normal pao2/fio2 ratio
## being on vent with low pao2/fio2 ratio

time: 17.3 ms


In [112]:
ratio_min_df.head()

,icustay_id,ratio,delta
0,200014,240.000000,17:11:00
1,200019,1980.952381,-1 days +12:17:00
2,200030,428.571429,-1 days +00:49:00
3,200065,98.000000,08:27:00
4,200078,140.000000,-1 days +19:08:00


time: 7.6 ms


In [113]:
final_pt_df2.head()

,subject_id,hadm_id,t_0_sc,icustay_id,ab_course,t_0,t_end_consec,ICU_admit,dod_hosp,dod_ssn,dod_delta,org_list,spec_type_list,Gram_pos,first_pos_else_neg_ssc,sc_result,final_bin
0,3281,113868,2115-09-14 19:25:00,240363.0,partial,2115-09-14,2115-09-16,2115-09-13 23:16:50,2116-03-06 00:00:00,2116-03-06 00:00:00,0 days 00:00:00.000000000,NaN,NaN,NaN,2115-09-14 19:25:00,negative,C_neg/A_partial
1,3277,163302,2123-07-24 09:30:00,220265.0,full,2123-07-24,2123-08-06,2123-07-21 13:04:46,NaN,NaN,NaN,NaN,NaN,NaN,2123-07-24 09:30:00,negative,C_neg/A_full
2,3276,111860,2168-08-31 00:00:00,224362.0,partial,2168-08-30,2168-08-30,2168-08-26 21:21:59,NaN,NaN,NaN,NaN,NaN,NaN,2168-08-31 00:00:00,negative,C_neg/A_partial
4,3267,134370,2193-08-28 09:30:00,278829.0,full,2193-08-28,2193-08-28,2193-08-26 12:28:29,2193-08-28 00:00:00,2193-08-28 00:00:00,0 days 00:00:00.000000000,NaN,NaN,NaN,2193-08-28 09:30:00,negative,C_neg/A_full
5,3266,156533,2137-04-03 23:30:00,271485.0,partial,2137-04-04,2137-04-04,2137-03-31 12:04:38,NaN,NaN,NaN,NaN,NaN,NaN,2137-04-03 23:30:00,negative,C_neg/A_partial


time: 37.9 ms


In [114]:
len(ratio_min_df)

3633

time: 3.38 ms


In [115]:
ratio_min_df= pd.merge(final_pt_df2[['icustay_id','hadm_id',time_var,]],ratio_min_df, left_on='icustay_id',right_on='icustay_id', how='left')
ratio_min_df= ratio_min_df.rename(index=str, columns={'ratio':'value'})

ratio_min_df.loc[ratio_min_df.loc[:,'value'].isnull(),'value']=476

ratio_min_df['uom']= 'PaO2/FIO2 ratio'
ratio_min_df['label']= 'PaO2/FIO2'

time: 21 ms


In [116]:
ratio_min_df.loc[:,'delta']=ratio_min_df.loc[:,'delta'].fillna(pd.to_timedelta('0 days'))


time: 5.47 ms


In [117]:
ratio_min_df['value'].describe()

count    8629.000000
mean      405.667593
std       170.946185
min        22.000000
25%       316.000000
50%       476.000000
75%       476.000000
max      2480.952381
Name: value, dtype: float64

time: 6.92 ms


In [118]:
save_df(ratio_min_df,'pafaRatio') # minimum pao2:fio2 ratio during a stay
del(ratio_min_df)

save_df(dfs['bg_all_nosummary_72'],'bg_all') #all bloodgas
del(dfs['bg_all_nosummary_72'])

save_df(bg_ART_nosummary_72,'bg_ART') #only ARTERIAL bloodgas
del(bg_ART_nosummary_72)

save_df(o2_flow_df,'o2_flow') #need to investigate this more
del(o2_flow_df)


time: 1.42 s


# RRT

In [119]:
#removing null values
dfs['rrt_merged_allpt_df'] = dfs['rrt_merged_allpt_df'].loc[dfs['rrt_merged_allpt_df']['rrt'].notnull(),:]
dfs['rrt_merged_allpt_df']['uom']='category'
dfs['rrt_merged_allpt_df']['delta']=pd.to_timedelta(
    pd.to_datetime(dfs['rrt_merged_allpt_df']['first_charttime'])-
    pd.to_datetime(dfs['rrt_merged_allpt_df'][time_var]),
    'days')
dfs['rrt_merged_allpt_df']['label']= 'rrt'
dfs['rrt_merged_allpt_df']=dfs['rrt_merged_allpt_df'].rename(index=str, columns={'rrt':'value'})

time: 23.1 ms


In [120]:
dfs['rrt_merged_allpt_df'].head()

,icustay_id,first_pos_else_neg_ssc,first_charttime,value,uom,delta,label
0,240363.0,2115-09-14 19:25:00,NaN,0.0,category,NaT,rrt
1,220265.0,2123-07-24 09:30:00,NaN,0.0,category,NaT,rrt
2,224362.0,2168-08-31 00:00:00,NaN,0.0,category,NaT,rrt
3,278829.0,2193-08-28 09:30:00,2193-08-27 09:30:00,1.0,category,-1 days,rrt
4,271485.0,2137-04-03 23:30:00,NaN,0.0,category,NaT,rrt


time: 12 ms


### converting to yes/no

In [121]:
dfs['rrt_merged_allpt_df'].head()

,icustay_id,first_pos_else_neg_ssc,first_charttime,value,uom,delta,label
0,240363.0,2115-09-14 19:25:00,NaN,0.0,category,NaT,rrt
1,220265.0,2123-07-24 09:30:00,NaN,0.0,category,NaT,rrt
2,224362.0,2168-08-31 00:00:00,NaN,0.0,category,NaT,rrt
3,278829.0,2193-08-28 09:30:00,2193-08-27 09:30:00,1.0,category,-1 days,rrt
4,271485.0,2137-04-03 23:30:00,NaN,0.0,category,NaT,rrt


time: 9.36 ms


In [122]:
dfs['rrt_merged_allpt_df']['icustay_id'].nunique() #

8629

time: 2.44 ms


In [123]:
rrt_yn= yn_convert(dfs['rrt_merged_allpt_df'],label_fill='rrt', time_var=time_var)

time: 38.6 ms


In [124]:
save_df(rrt_yn, 'rrt')
del(rrt_yn, dfs['rrt_merged_allpt_df'])

time: 199 ms


# GCS_72

In [125]:
dfs['gcs72_df']['uom']='GCS_score'

time: 1.35 ms


In [126]:
list(dfs['gcs72_df'])

['subject_id',
 'hadm_id',
 'icustay_id',
 'day',
 'mingcs',
 'gcsmotor',
 'gcsverbal',
 'gcseyes',
 'endotrachflag',
 'ICU_admit',
 'first_pos_else_neg_ssc',
 'approx_charttime',
 'admit_plus_day',
 'delta',
 'uom']

time: 6 ms


In [127]:
dfs['gcs72_df']['label']= 'mingcs'
dfs['gcs72_df']['uom']='gcs_score'
dfs['gcs72_df']=dfs['gcs72_df'].rename(index=str, columns={'mingcs':'value'})

time: 9.92 ms


In [128]:
dfs['gcs72_df']=dfs['gcs72_df'][['subject_id','hadm_id','icustay_id','delta','label','value',time_var,'uom']]

time: 2.2 ms


In [129]:
save_df(dfs['gcs72_df'], 'gcs')
del(dfs['gcs72_df'])

time: 17.6 ms


# SOFA_72
i'm going to remove all sofa variables except daily score, as we have other markers for those in our above data
i may later use this as qc check.
also added delta

In [130]:
dfs['sofa_df_72']['uom']='daily_sofa_score'

#adding day delta column
dfs['sofa_df_72']=dfs['sofa_df_72'].sort_values(['hadm_id','day',time_var]) #good
dfs['sofa_df_72']['day_rank']=dfs['sofa_df_72'].groupby('icustay_id')['day'].rank()
dfs['sofa_df_72']['delta']=pd.to_timedelta((dfs['sofa_df_72']['day_rank']-1), 'days')

time: 139 ms


In [131]:
dfs['sofa_df_72']['label']= 'daily_sofa'
dfs['sofa_df_72']=dfs['sofa_df_72'].rename(index=str, columns={'sofa':'value'})

time: 42.4 ms


In [132]:
dfs['sofa_df_72']= dfs['sofa_df_72'][['subject_id','hadm_id','icustay_id','delta','label','value',time_var,'uom']]

time: 4.22 ms


In [133]:
dfs['sofa_df_72'].head()

,subject_id,hadm_id,icustay_id,delta,label,value,first_pos_else_neg_ssc,uom
38824,54610,100003,209281,0 days,daily_sofa,9,2150-04-17 18:41:00,daily_sofa_score
38825,54610,100003,209281,1 days,daily_sofa,4,2150-04-17 18:41:00,daily_sofa_score
7894,9973,100020,282580,0 days,daily_sofa,0,2142-12-02 14:01:00,daily_sofa_score
36285,48539,100035,245719,0 days,daily_sofa,6,2115-02-22 09:36:00,daily_sofa_score
36286,48539,100035,245719,1 days,daily_sofa,5,2115-02-22 09:36:00,daily_sofa_score


time: 19.5 ms


In [134]:
save_df(dfs['sofa_df_72'], 'sofa')
del(dfs['sofa_df_72'])

time: 557 ms


# patient Demographic variables

#admissions
INSURANCE	VARCHAR(255)
LANGUAGE	VARCHAR(10)
RELIGION	VARCHAR(50)
MARITAL_STATUS	VARCHAR(50)
ETHNICITY	VARCHAR(200)

patients
GENDER	VARCHAR(5)

In [135]:
dfs['ethnicity_df']=(dfs['ethnicity_df'].loc[
    dfs['ethnicity_df'].loc[:,"subject_id"].isin(
        final_pt_df2['subject_id'].tolist()),:]).drop_duplicates(['subject_id','ethnicity'])

time: 22 ms


In [136]:
dfs['ethnicity_df']= dfs['ethnicity_df'][['subject_id','ethnicity']].sort_values('ethnicity', ascending=False).groupby('subject_id', as_index=False).first()


time: 13.3 ms


In [137]:
dfs['pt_info_df']=(dfs['pt_info_df'].loc[
    dfs['pt_info_df'].loc[:,"subject_id"].isin(
        final_pt_df2['subject_id'].tolist()),:]).drop_duplicates(['subject_id','gender'])

time: 7.37 ms


In [138]:
dfs['pt_info_df']= pd.merge(dfs['ethnicity_df'], dfs['pt_info_df'])

time: 6.52 ms


In [139]:
del(dfs['ethnicity_df'])

time: 710 µs


In [140]:
# date= '22112018'
# pd.DataFrame(pt_age_admin_df).to_csv(Path(
#     wd+'/data/processed/%s_pt_age.csv' %(date)))

age_df= pd.read_csv(Path(wd+'/data/processed/22112018_pt_age.csv'))


time: 260 ms


In [141]:
age_df=(age_df.loc[
    age_df.loc[:,"hadm_id"].isin(
        hadm_id),:])#.drop_duplicates(['subject_id','gender'])

time: 8.37 ms


In [142]:

age_df=pd.merge(age_df, final_pt_df2[['hadm_id','icustay_id',time_var]])
age_df=pd.merge(age_df[['subject_id','icustay_id','first_admit_age',time_var]],dfs['pt_info_df'])

time: 29.1 ms


In [143]:
age_df[age_df['first_admit_age']>89]['first_admit_age']=90

time: 57.4 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [144]:
age_df=pd.melt(age_df, id_vars=['icustay_id','subject_id',time_var])

time: 13 ms


In [145]:
age_df=age_df.rename(index=str, columns={'variable':'label'})

time: 15.2 ms


In [146]:
age_df['delta']=pd.to_timedelta('0 days')
age_df['uom']="N/A"
age_df.loc[age_df.loc[:,'label']=='first_admit_age','uom']='years'

time: 28.1 ms


In [147]:
age_df= age_df.loc[age_df.loc[:,"icustay_id"].isin(icustay_id),:]


time: 10.1 ms


In [148]:
age_df['label'].unique()

array(['first_admit_age', 'ethnicity', 'gender'], dtype=object)

time: 3.09 ms


In [149]:
###using regular expressions to reduce the # of ethinicities

age_df.loc[(age_df.loc[:,"label"]=='ethnicity') & (age_df.loc[:,"value"].str.contains(r'.*?BLACK')),'value']="black"
age_df.loc[(age_df.loc[:,"label"]=='ethnicity') & (age_df.loc[:,"value"].str.contains(r'.*?HISPANIC|PORTUGUESE')),'value']="hispanic"
age_df.loc[(age_df.loc[:,"label"]=='ethnicity') & (age_df.loc[:,"value"].str.contains(r'.*?WHITE')),'value']="white/nonhispanic"
age_df.loc[(age_df.loc[:,"label"]=='ethnicity') & (age_df.loc[:,"value"].str.contains(r'.*?ASIAN')),'value']='asian'
age_df.loc[(age_df.loc[:,"label"]=='ethnicity') & (age_df.loc[:,"value"].str.contains(r'(UNKNOWN|MULTI|UNABLE|DECLINE|OTHER)')),'value']='unknown/other'
age_df.loc[(age_df.loc[:,"label"]=='ethnicity') & (age_df.loc[:,"value"].str.contains(r'[AZ]+')),'value']="unknown/other" #lumping all other low n values into other


time: 292 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  import sys


In [150]:
age_df.loc[age_df.loc[:,"label"]=='ethnicity','value'].value_counts()

white/nonhispanic    6180
unknown/other        1023
black                 888
hispanic              306
asian                 232
Name: value, dtype: int64

time: 23.8 ms


In [151]:
#date= '22102018'
save_df(age_df, 'pt_info')

del(age_df,dfs['pt_info_df'] )

time: 527 ms
